In [13]:
import os
import shutil

def restructure_dataset(source_root, target_root):
    # Create the main target directory
    os.makedirs(target_root, exist_ok=True)

    # Iterate through each class (01, 02, 03)
    for class_name in os.listdir(source_root):
        class_path = os.path.join(source_root, class_name)
        if not os.path.isdir(class_path):
            continue

        # Create class directory in the target
        target_class_path = os.path.join(target_root, f"class_{class_name}")
        os.makedirs(target_class_path, exist_ok=True)

        # Create 'test' and 'train' directories
        os.makedirs(os.path.join(target_class_path, 'test', 'anomaly'), exist_ok=True)
        os.makedirs(os.path.join(target_class_path, 'test', 'good'), exist_ok=True)
        os.makedirs(os.path.join(target_class_path, 'train', 'good'), exist_ok=True)

        # Copy files for test split
        source_test_path = os.path.join(class_path, 'test')
        if os.path.exists(source_test_path):
            for condition in ['ok', 'ko']:
                source_condition_path = os.path.join(source_test_path, condition)
                if os.path.exists(source_condition_path):
                    target_condition = 'good' if condition == 'ok' else 'anomaly'
                    target_condition_path = os.path.join(target_class_path, 'test', target_condition)
                    for file in os.listdir(source_condition_path):
                        source_file = os.path.join(source_condition_path, file)
                        target_file = os.path.join(target_condition_path, file)
                        shutil.copy2(source_file, target_file)

        # Copy files for train split (only 'ok' images)
        source_train_path = os.path.join(class_path, 'train', 'ok')
        if os.path.exists(source_train_path):
            target_train_path = os.path.join(target_class_path, 'train', 'good')
            for file in os.listdir(source_train_path):
                source_file = os.path.join(source_train_path, file)
                target_file = os.path.join(target_train_path, file)
                shutil.copy2(source_file, target_file)

    print("Dataset restructuring completed.")

# Usage
source_root = '/teamspace/studios/this_studio/BTAD/BTech_Dataset_transformed'
target_root = '/teamspace/studios/this_studio/BTAD_Dataset'
restructure_dataset(source_root, target_root)

Dataset restructuring completed.


In [14]:
import os
from collections import defaultdict

def count_images(path):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    return sum(1 for file in os.listdir(path) if file.lower().endswith(image_extensions))

def verify_dataset(source_root, target_root):
    source_counts = defaultdict(lambda: defaultdict(int))
    target_counts = defaultdict(lambda: defaultdict(int))

    # Count images in source dataset
    for class_folder in os.listdir(source_root):
        class_path = os.path.join(source_root, class_folder)
        if not os.path.isdir(class_path):
            continue

        for split in ['test', 'train']:
            split_path = os.path.join(class_path, split)
            if not os.path.exists(split_path):
                continue

            for condition in os.listdir(split_path):
                condition_path = os.path.join(split_path, condition)
                if condition == 'ok':
                    source_counts[class_folder][f"{split}/good"] += count_images(condition_path)
                elif condition == 'ko':
                    source_counts[class_folder][f"{split}/anomaly"] += count_images(condition_path)

    # Count images in target dataset
    for class_folder in os.listdir(target_root):
        class_path = os.path.join(target_root, class_folder)
        if not os.path.isdir(class_path):
            continue

        for split in ['test', 'train']:
            split_path = os.path.join(class_path, split)
            if not os.path.exists(split_path):
                continue

            for condition in ['good', 'anomaly']:
                condition_path = os.path.join(split_path, condition)
                if os.path.exists(condition_path):
                    target_counts[class_folder][f"{split}/{condition}"] += count_images(condition_path)

    # Print results and verify
    print("Source dataset counts:")
    print_counts(source_counts)

    print("\nTarget dataset counts:")
    print_counts(target_counts)

    print("\nVerification results:")
    verify_counts(source_counts, target_counts)

def print_counts(counts):
    for class_name, categories in counts.items():
        print(f"\n{class_name}:")
        for category, count in categories.items():
            print(f"  {category}: {count}")

def verify_counts(source_counts, target_counts):
    all_match = True
    for class_name in source_counts.keys():
        for category in source_counts[class_name].keys():
            source_count = source_counts[class_name][category]
            target_count = target_counts[class_name.replace('0', 'class_0')][category]
            if source_count != target_count:
                print(f"Mismatch in {class_name} - {category}: Source: {source_count}, Target: {target_count}")
                all_match = False

    if all_match:
        print("All counts match between source and target datasets.")
    else:
        print("There are mismatches between source and target datasets.")

# Usage
source_root = 'BTech_Dataset_transformed'
target_root = '/teamspace/studios/this_studio/BTAD_Dataset'
verify_dataset(source_root, target_root)

Source dataset counts:

03:
  test/good: 400
  test/anomaly: 41
  train/good: 1000

02:
  test/good: 30
  test/anomaly: 200
  train/good: 399

01:
  test/good: 21
  test/anomaly: 49
  train/good: 400

Target dataset counts:

class_01:
  test/good: 21
  test/anomaly: 49
  train/good: 400

class_02:
  test/good: 30
  test/anomaly: 200
  train/good: 399

class_03:
  test/good: 400
  test/anomaly: 41
  train/good: 1000

Verification results:
All counts match between source and target datasets.


## Training on BTAD

### Class 1

In [3]:
!python main.py

Training model for class: class_01
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
Epoch: 0.0 	 train loss: -0.1013
Epoch: 0.1 	 train loss: -1.6711
Epoch: 0.2 	 train loss: -1.8539
Epoch: 0.3 	 train loss: -1.1872
Epoch: 0.4 	 train loss: -1.6930
Epoch: 0.5 	 train loss: -1.1628
Epoch: 0.6 	 train loss: -1.6865
Epoch: 0.7 	 train loss: -1.7110

Compute lo

### Class 2 And 3

In [2]:
cd differnet-master/

/teamspace/studios/this_studio/differnet-master


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!python main.py

Training model for class: class_02
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)

Train epoch 0
Epoch: 0.0 	 train loss: -0.3687
Epoch: 0.1 	 train loss: -1.2872
Epoch: 0.2 	 train loss: -1.5397
Epoch: 0.3 	 train loss: -1.7760
Epoch: 0.4 	 train loss: -1.9131
Epoch: 0.5 	 train loss: -2.0197
Epoch: 0.6 	 train loss: -2.0874
Epoch: 0.7 	 train loss: -2.1646

Compute lo